<https://github.com/PolymathicAI/xVal>


In [1]:
from datetime import datetime as dt
from typing import Union

import icecream
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import orbax
import orbax.checkpoint
import pandas as pd
from flax.struct import dataclass
from flax.training import orbax_utils, train_state
from icecream import ic
from jax import random
from jax.tree_util import tree_flatten
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm, trange
from transformers import BertTokenizerFast, FlaxBertModel

import hephaestus as hp
import hephaestus.models.time_series_decoder_training as tsd
import hashlib
import random as py_random
import nltk
from nltk.corpus import words

# Predefined list of words


# Download the words corpus if not already downloaded
nltk.download("words")

icecream.install()
ic_disable = False  # Global variable to disable ic
if ic_disable:
    ic.disable()
ic.configureOutput(includeContext=True, contextAbsPath=True)
pd.options.mode.copy_on_write = True

[nltk_data] Downloading package words to
[nltk_data]     /Users/kailukowiak/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
word_list = words.words()

In [3]:
# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
model = FlaxBertModel.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# Get the embeddings matrix
embeddings = model.params["embeddings"]["word_embeddings"]["embedding"]

# Now you can access specific embeddings like this:
# For example, to get embeddings for tokens 23, 293, and 993:
selected_embeddings = jnp.take(embeddings, jnp.array([23, 293, 993]), axis=0)

# If you want to get embeddings for specific words:
words = ["hello", "world", "example"]
tokens = tokenizer.convert_tokens_to_ids(words)
word_embeddings = jnp.take(embeddings, jnp.array(tokens), axis=0)
word_embeddings.shape

Some weights of FlaxBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: {('pooler', 'dense', 'kernel'), ('pooler', 'dense', 'bias')}
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(3, 768)

In [4]:
df = pd.read_csv("data/Koinly/koinly_example.csv")
df.rename(
    lambda x: x.lower().replace(" ", "_").replace("(", "").replace(")", ""),
    axis=1,
    inplace=True,
)

df[df.select_dtypes(include=["object"]).columns] = df.select_dtypes(
    include=["object"]
).fillna("[missing]")

df[df.select_dtypes(include=["number"]).columns] = df.select_dtypes(
    include=["number"]
).fillna(0.0)
df = df.dropna(axis=0, how="all")
df = df.dropna(axis=1, how="all")
df = df.drop(columns=["date"])

df.head()

,type,tag,sending_wallet,sent_amount,sent_currency,sent_cost_basis,receiving_wallet,received_amount,received_currency,received_cost_basis,fee_amount,fee_currency,gain_usd,net_value_usd,fee_value_usd,txsrc,txdest,txhash,description
0,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.369250,NOS,0.261250,0.0,[missing],0.000000,0.261250,0.0,[missing],[missing],269CiLPaFK55QqiVbsJupN6BSPUHQ3x7kN6iHPhSMV2NDw...,[missing]
1,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.001721,SOL,0.175936,0.0,[missing],0.000000,0.175936,0.0,[missing],[missing],269CiLPaFK55QqiVbsJupN6BSPUHQ3x7kN6iHPhSMV2NDw...,[missing]
2,crypto_withdrawal,[missing],Solana (SOL) 10,0.001736,SOL,0.180755,[missing],0.000000,[missing],0.000000,0.0,[missing],-0.003286,0.177470,0.0,[missing],[missing],FaDbep4DQ7pJgHdvAoxZ7wvkfjHxLn99aZ8oT1EyaNE6L3...,[missing]
3,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.373500,NOS,0.264257,0.0,[missing],0.000000,0.264257,0.0,[missing],[missing],4jfNX5Ja3CnxMLHZCevfLw4k1A2SQxrMh6r2K3mZKz5QfQ...,[missing]
4,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.001721,SOL,0.175936,0.0,[missing],0.000000,0.175936,0.0,[missing],[missing],4jfNX5Ja3CnxMLHZCevfLw4k1A2SQxrMh6r2K3mZKz5QfQ...,[missing]


In [5]:
datetime_cols = ["created_at", "updated_at", "date"]


def enrich_datetimes(df: pd.DataFrame, col: str):
    df[col] = pd.to_datetime(df[col], utc=True)

    df[f"{col}_year"] = df[col].dt.year
    df[f"{col}_month_sin"] = np.sin(2 * np.pi * df[col].dt.month / 12)
    df[f"{col}_month_cos"] = np.cos(2 * np.pi * df[col].dt.month / 12)
    df[f"{col}_day_sin"] = np.sin(2 * np.pi * df[col].dt.day / 31)
    df[f"{col}_day_cos"] = np.cos(2 * np.pi * df[col].dt.day / 31)
    df[f"{col}_hour_sin"] = np.sin(2 * np.pi * df[col].dt.hour / 24)
    df[f"{col}_hour_cos"] = np.cos(2 * np.pi * df[col].dt.hour / 24)
    df[f"{col}_minute_sin"] = np.sin(2 * np.pi * df[col].dt.minute / 60)
    df[f"{col}_minute_cos"] = np.cos(2 * np.pi * df[col].dt.minute / 60)
    df[f"{col}_second_sin"] = np.sin(2 * np.pi * df[col].dt.second / 60)
    df[f"{col}_second_cos"] = np.cos(2 * np.pi * df[col].dt.second / 60)

    return df.drop(columns=[col])


# for col in datetime_cols:
#     df = enrich_datetimes(df, col)

In [6]:
# Use the words corpus from nltk
# words_list = words.words()


def hash_to_words(crypto_hash: Union[str, np.nan]) -> Union[str, np.nan]:
    if crypto_hash is np.nan:
        return np.nan
    # Create a hash of the input
    hash_object = hashlib.sha256(crypto_hash.encode())
    hash_digest = hash_object.hexdigest()

    # Use the hash to select three words from the list
    py_random.seed(hash_digest)
    selected_words = py_random.sample(word_list, 3)

    return "hash " + " ".join(selected_words)


# Example usage
crypto_hash = "269CiLPaFK55QqiVbsJupN6BSPUHQ3x7kN6iHPhSMV2NDwHM2EhnwQ6hE6FEvbix6AVN2PLUMQyyhrKr2y514dRB"
print(hash_to_words(crypto_hash))

hash rabies placophoran isagogically


In [7]:
df.head()

,type,tag,sending_wallet,sent_amount,sent_currency,sent_cost_basis,receiving_wallet,received_amount,received_currency,received_cost_basis,fee_amount,fee_currency,gain_usd,net_value_usd,fee_value_usd,txsrc,txdest,txhash,description
0,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.369250,NOS,0.261250,0.0,[missing],0.000000,0.261250,0.0,[missing],[missing],269CiLPaFK55QqiVbsJupN6BSPUHQ3x7kN6iHPhSMV2NDw...,[missing]
1,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.001721,SOL,0.175936,0.0,[missing],0.000000,0.175936,0.0,[missing],[missing],269CiLPaFK55QqiVbsJupN6BSPUHQ3x7kN6iHPhSMV2NDw...,[missing]
2,crypto_withdrawal,[missing],Solana (SOL) 10,0.001736,SOL,0.180755,[missing],0.000000,[missing],0.000000,0.0,[missing],-0.003286,0.177470,0.0,[missing],[missing],FaDbep4DQ7pJgHdvAoxZ7wvkfjHxLn99aZ8oT1EyaNE6L3...,[missing]
3,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.373500,NOS,0.264257,0.0,[missing],0.000000,0.264257,0.0,[missing],[missing],4jfNX5Ja3CnxMLHZCevfLw4k1A2SQxrMh6r2K3mZKz5QfQ...,[missing]
4,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.001721,SOL,0.175936,0.0,[missing],0.000000,0.175936,0.0,[missing],[missing],4jfNX5Ja3CnxMLHZCevfLw4k1A2SQxrMh6r2K3mZKz5QfQ...,[missing]


In [8]:
hash_columns = [i for i in df.columns if "tx" in i]
hash_columns

for col in hash_columns:
    df[col] = df[col].apply(hash_to_words)

# Drop hash columns for testing
df = df.drop(columns=[i for i in df.columns if "tx" in i])

In [9]:
df.dtypes

type                    object
tag                     object
sending_wallet          object
sent_amount            float64
sent_currency           object
sent_cost_basis        float64
receiving_wallet        object
received_amount        float64
received_currency       object
received_cost_basis    float64
fee_amount             float64
fee_currency            object
gain_usd               float64
net_value_usd          float64
fee_value_usd          float64
description             object
dtype: object

In [10]:
df.index.unique()

RangeIndex(start=0, stop=12296, step=1)

In [11]:
df["idx"] = df.index // 32
df.index = df.index % 32
# df = df.loc[
#     :,
#     [
#         "idx",
#         # "type",
#         # "sending_wallet",
#         # "sent_amount",
#         # "sent_currency",
#         # "sent_cost_basis",
#     ],
# ]
# df["test_index"] = df.index
# df["test_index_2"] = df.index * 2
# # make new column for if test_index is even or odd
# df["even_odd"] = df["test_index"] % 2
# df["even_odd_str"] = df["even_odd"].apply(lambda x: "even" if x == 0 else "odd")
# # Make a new column for sin and cos for each index
# df["sin"] = np.sin(2 * np.pi * df["test_index"])
# df.head()
df.head(45)

,type,tag,sending_wallet,sent_amount,sent_currency,sent_cost_basis,receiving_wallet,received_amount,received_currency,received_cost_basis,fee_amount,fee_currency,gain_usd,net_value_usd,fee_value_usd,description,idx
0,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.369250,NOS,0.261250,0.0,[missing],0.000000,0.261250,0.0,[missing],0
1,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.001721,SOL,0.175936,0.0,[missing],0.000000,0.175936,0.0,[missing],0
2,crypto_withdrawal,[missing],Solana (SOL) 10,0.001736,SOL,0.180755,[missing],0.000000,[missing],0.000000,0.0,[missing],-0.003286,0.177470,0.0,[missing],0
3,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.373500,NOS,0.264257,0.0,[missing],0.000000,0.264257,0.0,[missing],0
4,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.001721,SOL,0.175936,0.0,[missing],0.000000,0.175936,0.0,[missing],0
5,crypto_withdrawal,[missing],Solana (SOL) 10,0.001736,SOL,0.180755,[missing],0.000000,[missing],0.000000,0.0,[missing],-0.003286,0.177470,0.0,[missing],0
6,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.294500,NOS,0.208363,0.0,[missing],0.000000,0.208363,0.0,[missing],0
7,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.001721,SOL,0.175936,0.0,[missing],0.000000,0.175936,0.0,[missing],0
8,crypto_withdrawal,[missing],Solana (SOL) 10,0.001736,SOL,0.180755,[missing],0.000000,[missing],0.000000,0.0,[missing],-0.003286,0.177470,0.0,[missing],0
9,crypto_deposit,[missing],[missing],0.000000,[missing],0.000000,Solana (SOL) 10,0.282500,NOS,0.202806,0.0,[missing],0.000000,0.202806,0.0,[missing],0


In [12]:
df.head(1000).to_parquet("data/konly_remixed.parquet", index=False)

In [13]:
# Get train test split at 80/20
time_series_config = hp.TimeSeriesConfig.generate(df=df)
train_idx = int(df.idx.max() * 0.8)
train_df = df.loc[df.idx < train_idx].copy()
test_df = df.loc[df.idx >= train_idx].copy()
# del df
train_ds = hp.TimeSeriesDS(train_df, time_series_config)
test_ds = hp.TimeSeriesDS(test_df, time_series_config)
len(train_ds), len(test_ds)

(307, 78)

In [14]:
def make_batch(ds: hp.TimeSeriesDS, start: int, length: int):
    numeric = []
    categorical = []
    for i in range(start, length + start):
        numeric.append(ds[i][0])
        categorical.append(ds[i][1])
    # print index of None values
    return {"numeric": jnp.array(numeric), "categorical": jnp.array(categorical)}


batch = make_batch(train_ds, 0, 4)
# batch

In [15]:
multiplier = 4
time_series_regressor = hp.TimeSeriesDecoder(
    time_series_config, d_model=512, n_heads=8 * multiplier
)

In [16]:
key = random.PRNGKey(0)
init_key, dropout_key = random.split(key)
vars = time_series_regressor.init(
    {"params": init_key, "dropout": dropout_key},
    batch["numeric"],
    categorical_inputs=batch["categorical"].astype(jnp.int32),
    deterministic=False,
)
dropout_key, original_dropout_key = random.split(dropout_key)

ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:444 in process_numeric()
    numeric_embedding.shape: (512,)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:447 in process_numeric()
    numeric_embedding.shape: (4, 8, 32, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:455 in process_numeric()
    numeric_embedding.shape: (4, 8, 32, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:519 in combine_inputs()
    numeric.value_embeddings.shape: (4, 8, 32, 512)
    categorical.value_embeddings.shape: (4, 8, 32, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:617 in __call__()
    combined_inputs.value_embeddings.shape: (4, 16, 32, 512)
    combined_inputs.column_embeddings.shape: (4, 16, 32, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:314 in __call__()
    "Transformer Block": 'Transformer Block'
    q.shape: 

In [17]:
x = time_series_regressor.apply(
    vars,
    batch["numeric"],
    batch["categorical"].astype(jnp.int32),
    deterministic=False,
    rngs={"dropout": dropout_key},
)
print(x.get("numeric_out").shape)
# Check if categorical input is None and print None or it's shape
print(x.get("categorical_out").shape if x.get("categorical_out") is not None else None)

ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:444 in process_numeric()
    numeric_embedding.shape: (512,)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:447 in process_numeric()
    numeric_embedding.shape: (4, 8, 32, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:455 in process_numeric()
    numeric_embedding.shape: (4, 8, 32, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:519 in combine_inputs()
    numeric.value_embeddings.shape: (4, 8, 32, 512)
    categorical.value_embeddings.shape: (4, 8, 32, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:617 in __call__()
    combined_inputs.value_embeddings.shape: (4, 16, 32, 512)
    combined_inputs.column_embeddings.shape: (4, 16, 32, 512)
ic| /Users/kailukowiak/Hephaestus/hephaestus/models/time_series_decoder.py:314 in __call__()
    "Transformer Block": 'Transformer Block'
    q.shape: 

(4, 8, 32)
(4, 8, 32, 205)


In [18]:
x["categorical_out"].shape

(4, 8, 32, 205)

In [19]:
def calculate_memory_footprint(params):
    """Calculate total memory footprint of JAX model parameters and total
    number of parameters."""
    total_bytes = 0
    # Flatten the parameter tree structure into a list of arrays
    flat_params, _ = tree_flatten(params)
    for param in flat_params:
        # Calculate bytes: number of elements * size of each element
        bytes_per_param = param.size * param.dtype.itemsize
        total_bytes += bytes_per_param
    return total_bytes


def count_parameters(params):
    return sum(jnp.prod(jnp.array(p.shape)) for p in jax.tree_util.tree_leaves(params))


mem = calculate_memory_footprint(vars)
total_params = count_parameters(vars)


print(f"Memory of custom: {mem / 1e6:.2f} MB with {total_params:,} parameters")

Memory of custom: 52.33 MB with 13,083,229 parameters


In [20]:
ic.disable()

In [21]:
mts_root_key = random.PRNGKey(44)
mts_main_key, ts_params_key, ts_data_key = random.split(mts_root_key, 3)

causal_mask = False


batch_size = 2

state = tsd.create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)

In [22]:
@jax.jit
def my_train_step(
    state: train_state.TrainState,
    numerical_inputs,
    categorical_inputs,
    base_key,
):
    return tsd.train_step(
        state,
        numerical_inputs,
        categorical_inputs,
        base_key,
        input_offset=0,
    )

In [23]:
# causal_mask = False

In [24]:
writer_name = "New_Index"

writer_time = dt.now().strftime("%Y-%m-%dT%H:%M:%S")
model_name = writer_time + writer_name
train_summary_writer = SummaryWriter("runs/" + model_name)


test_set_key = random.PRNGKey(444)

batch_size = 4
train_data_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

# train_data_loader = DataLoader(train_ds, batch_size=256 // 2, shuffle=True)
# test_data_loader = DataLoader(test_ds, batch_size=256 // 2, shuffle=True)

batch_count = 0
base_key = random.PRNGKey(42)

# Disable IC for training
max_iters = None
ic.disable()
for j in trange(3, desc=f"epochs for {train_summary_writer.log_dir}"):
    # arrs = train_data_loader()
    for i in tqdm(train_data_loader, leave=False, desc="batches"):
        # for i in trange(len(pre_train) // batch_size, leave=False):
        # for i in trange(len(pre_train) // batch_size //10, leave=False):
        # batch = make_batch(train_ds, i[0], 4)
        state, loss, base_key = my_train_step(
            state,
            jnp.array(i[0]),
            jnp.array(i[1]),
            base_key,
            # input_offset=0,
            # causal_mask=causal_mask,
        )
        if jnp.isnan(loss):
            raise ValueError("Nan Value in loss, stopping")
        batch_count += 1

        if batch_count % 1 == 0:
            train_summary_writer.add_scalar(
                "loss/loss", np.array(loss.item()), batch_count
            )
        if batch_count % 10 == 0:
            numeric_eval, categorical_eval = next(iter(test_data_loader))
            test_loss, base_key = tsd.eval_step(
                state,
                jnp.array(numeric_eval),
                jnp.array(categorical_eval),
                base_key,
                # causal_mask=causal_mask,
            )
            train_summary_writer.add_scalar(
                "loss/test_loss", np.array(test_loss.item()), batch_count
            )
            train_summary_writer.flush()
        # if batch_count > 200:
        #     break
        if not max_iters:
            continue
        else:
            if batch_count > max_iters:
                break

train_summary_writer.close()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


epochs for runs/2024-10-24T22:35:56New_Index:   0%|          | 0/3 [00:00<?, ?it/s]

batches:   0%|          | 0/77 [00:00<?, ?it/s]

batches:   0%|          | 0/77 [00:00<?, ?it/s]

batches:   0%|          | 0/77 [00:00<?, ?it/s]

In [25]:
loss

Array(15.14869, dtype=float32)

In [26]:
print("Done")

Done


In [27]:
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()

In [28]:
ckpt = {"model": state, "step": batch_count}


checkpoint_dir = f"checkpoints/{model_name}"
checkpoint_dir = os.path.abspath(checkpoint_dir)

# os.makedirs(checkpoint_dir, exist_ok=True)

orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
save_args = orbax_utils.save_args_from_target(ckpt)
orbax_checkpointer.save(checkpoint_dir, ckpt, save_args=save_args)

NameError: name 'os' is not defined

In [27]:
# if model_name is None:
if True:
    all_checkpoints = os.listdir("checkpoints/")
    model_name = all_checkpoints
    all_checkpoints = sorted(all_checkpoints)
    model_name = all_checkpoints[-1]
    checkpoint_dir = f"checkpoints/{model_name}"
    checkpoint_dir = os.path.abspath(checkpoint_dir)

new_checkpoint = orbax_checkpointer.restore(checkpoint_dir)
new_state = tsd.create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)
new_state = new_state.replace(params=new_checkpoint["model"]["params"])

/opt/homebrew/Caskroom/miniconda/base/envs/hephaestus/lib/python3.11/site-packages/orbax/checkpoint/type_handlers.py:1552: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


In [28]:
numeric_inputs, categorical_inputs = train_ds[0]
numeric_inputs = jnp.array([numeric_inputs])
categorical_inputs = jnp.array([categorical_inputs])
test_results = state.apply_fn(
    {"params": state.params},
    # jnp.array(i[0]),
    # jnp.array(i[1]),
    numeric_inputs,
    categorical_inputs,
    deterministic=True,
    causal_mask=False,
)
test_results["numeric_out"].shape

(1, 4, 59)

In [29]:
@dataclass
class Results:
    numeric_out: jnp.array
    categorical_out: jnp.array
    numeric_inputs: jnp.array
    categorical_inputs: jnp.array


def return_results(state, dataset, idx=0, mask_start: int = None):
    numeric_inputs, categorical_inputs = dataset[idx]
    if mask_start:
        numeric_inputs = numeric_inputs[:, :mask_start]
        categorical_inputs = categorical_inputs[:, :mask_start]
    numeric_inputs = jnp.array([numeric_inputs])
    categorical_inputs = jnp.array([categorical_inputs])
    out = state.apply_fn(
        {"params": state.params},
        # hp.mask_tensor(jnp.array([train_ds[0]]), dataset, prng_key=key),
        numeric_inputs=numeric_inputs,
        categorical_inputs=categorical_inputs,
        deterministic=True,
        causal_mask=causal_mask,
    )
    numeric_out, categorical_out = out["numeric_out"], out["categorical_out"]
    return Results(numeric_out, categorical_out, numeric_inputs, categorical_inputs)


x = return_results(state, train_ds, 0)
x.categorical_out.shape

(1, 1, 59, 12)

In [30]:
causal_mask = False
causal_mask = True


def process_results(arr: jnp.array, col_names: list, config: hp.TimeSeriesConfig):
    arr = jnp.squeeze(arr)
    if arr.ndim == 3:
        # Check if there is a logit array for example if there are 3 dims then the
        # last dim is the logit array. We need to get the argmax of the last dim
        # to get the actual values of the array and replace the logit array with the
        # actual values
        arr = jnp.argmax(arr, axis=-1)
    df = pd.DataFrame(arr.T)
    df.columns = col_names
    return df


@dataclass
class DFComparison:
    input_df: pd.DataFrame
    output_df: pd.DataFrame


def show_results_df(
    state, time_series_config, dataset, idx: int = 0, mask_start: int = None
):
    results = return_results(state, dataset, idx=idx, mask_start=mask_start)

    input_categorical = process_results(
        results.categorical_inputs,
        time_series_config.categorical_col_tokens,
        time_series_config,
    )
    input_numeric = process_results(
        results.numeric_inputs,
        time_series_config.numeric_col_tokens,
        time_series_config,
    )
    output_categorical = process_results(
        results.categorical_out,
        time_series_config.categorical_col_tokens,
        time_series_config,
    )
    output_numeric = process_results(
        results.numeric_out, time_series_config.numeric_col_tokens, time_series_config
    )
    input_df = pd.concat([input_categorical, input_numeric], axis=1)
    output_df = pd.concat([output_categorical, output_numeric], axis=1)

    return DFComparison(input_df, output_df)


df_comp = show_results_df(
    state=state, time_series_config=time_series_config, dataset=train_ds, idx=0
)

ValueError: Length mismatch: Expected axis has 59 elements, new values have 1 elements

In [31]:
df_comp.output_df.loc[:, time_series_config.categorical_col_tokens].tail()

,type,tag,sending_wallet,sent_currency,receiving_wallet,received_currency,fee_currency,description
95,172,185,113,176,176,185,8,185
96,188,185,185,172,113,176,181,185
97,188,185,185,172,113,176,84,185
98,172,185,113,176,176,185,8,185
99,188,185,185,172,113,176,181,185


In [32]:
df_comp.output_df.loc[:, time_series_config.categorical_col_tokens].tail()

,type,tag,sending_wallet,sent_currency,receiving_wallet,received_currency,fee_currency,description
95,172,185,113,176,176,185,8,185
96,188,185,185,172,113,176,181,185
97,188,185,185,172,113,176,84,185
98,172,185,113,176,176,185,8,185
99,188,185,185,172,113,176,181,185


In [33]:
def plot_planets(df_pred: pd.DataFrame, df_actual: pd.DataFrame, column: str, offset=0):
    plt.figure(figsize=(15, 10))
    plt.plot(df_pred[column], label="Autogregressive")
    plt.plot(df_actual[column], label="Actual")
    plt.title(f"{column} Predictions")
    plt.legend()
    # Show ticks and grid lines every 1 step
    plt.xticks(np.arange(0, len(df_pred), 1))
    plt.grid()
    # add black line at 0 on the y axis to show the difference
    plt.axhline(0, color="black")
    plt.show()

In [34]:
jnp.array([True, True, False, False, True]).shape

(5,)

In [35]:
x = jnp.ones((5, 20))
print(x.shape)
xx = x.at[jnp.array([True, True, False, False, True]), :].set(0)
xx

(5, 20)


Array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32)

In [36]:
@dataclass
class AutoRegressiveResults:
    numeric_inputs: jnp.array
    categorical_inputs: jnp.array

    @classmethod
    def from_ds(cls, ds: hp.TimeSeriesDS, idx: int, stop_idx: int = 10):
        inputs = ds[idx]
        numeric_inputs = inputs[0][:, :stop_idx]
        categorical_inputs = inputs[1][:, :stop_idx]
        return cls(numeric_inputs, categorical_inputs)


def auto_regressive_predictions(
    state: train_state.TrainState,
    inputs: AutoRegressiveResults,
) -> jnp.array:
    numeric_inputs = inputs.numeric_inputs
    categorical_inputs = inputs.categorical_inputs
    # get the first row that contains all nan vales
    # if nan_rows_start >= stop_idx:
    #     return inputs
    # numeric_inputs = inputs.numeric_inputs
    # categorical_inputs = inputs.categorical_inputs
    numeric_nan_columns = jnp.isnan(numeric_inputs).all(axis=1)
    categorical_nan_columns = jnp.isnan(categorical_inputs).all(axis=1)
    outputs = state.apply_fn(
        {"params": state.params},
        numeric_inputs=jnp.array([numeric_inputs]),
        categorical_inputs=jnp.array([categorical_inputs]),
        deterministic=True,
        causal_mask=False,
    )
    numeric_out = jnp.squeeze(outputs["numeric_out"])
    categorical_out = jnp.squeeze(outputs["categorical_out"])
    categorical_out = jnp.argmax(categorical_out, axis=-1)

    final_numeric_row = np.array(numeric_out[:, -1])
    final_numeric_row = final_numeric_row[:, None]  # New axis

    final_categorical_row = np.array(categorical_out[:, -1])
    final_categorical_row = final_categorical_row[:, None]  # New axis
    numeric_inputs = jnp.concatenate([numeric_inputs, final_numeric_row], axis=1)
    categorical_inputs = jnp.concatenate(
        [categorical_inputs, final_categorical_row], axis=1
    )
    numeric_inputs = numeric_inputs.at[jnp.array(numeric_nan_columns)].set(jnp.nan)
    categorical_inputs = categorical_inputs.at[jnp.array(categorical_nan_columns)].set(
        jnp.nan
    )
    inputs = (numeric_inputs, categorical_inputs)

    return inputs
    # return auto_regressive_predictions(state, inputs, stop_idx)

In [37]:
test_inputs = AutoRegressiveResults.from_ds(train_ds, 0, 10)
# inputs_test = train_ds[0]
# test_numeric = inputs_test[0]
# test_categorical = inputs_test[1]
# print(inputs_test.shape)
for i in trange(21):
    inputs_test = auto_regressive_predictions(state, test_inputs)

# x = auto_regressive_predictions(state, test_ds[0], 10)

  0%|          | 0/21 [00:00<?, ?it/s]

In [38]:
# res = show_results_df(state, train_df, test_ds, idx=0, mask_start=30)

# plot_planets(res["pred"], res["actual_masked"], "planet2_x", offset=0)